In [ ]:
import os

REPO_NAME = "technical-challenge"
if REPO_NAME not in os.getcwd():
  if not os.path.exists(REPO_NAME):
    !git clone https://github.com/enzoferrari1/{REPO_NAME}.git
  os.chdir(REPO_NAME)

Cloning into 'technical-challenge'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 13 (delta 1), reused 5 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (13/13), 1.75 MiB | 3.61 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px

In [ ]:
import pandas as pd

data_input = pd.read_csv("data/people.csv")
data_descriptions = pd.read_csv("data/descriptions.csv")
data_target = pd.read_csv("data/salary.csv")

# data_input.drop(columns=["id"], inplace=True)
# data_descriptions.drop(columns=["id"], inplace=True)
# data_target.drop(columns=["id"], inplace=True)

data_input

,id,Age,Gender,Education Level,Job Title,Years of Experience
0,0,32.0,Male,Bachelor's,Software Engineer,5.0
1,1,28.0,Female,Master's,Data Analyst,3.0
2,2,45.0,Male,PhD,Senior Manager,15.0
3,3,36.0,Female,Bachelor's,Sales Associate,7.0
4,4,52.0,Male,Master's,Director,20.0
...,...,...,...,...,...,...
370,370,35.0,Female,Bachelor's,Senior Marketing Analyst,8.0
371,371,43.0,Male,Master's,Director of Operations,19.0
372,372,29.0,Female,Bachelor's,Junior Project Manager,2.0
373,373,34.0,Male,Bachelor's,Senior Operations Coordinator,7.0


In [ ]:
data = pd.merge(data_input, data_descriptions, on="id")
data.dropna(subset=["Job Title", "Description"], inplace=True)

# Feature Engineering
Como nuestro objetivo es llegar a una manera de entender

In [ ]:
# Sentences we want sentence embeddings for
sentences = [str(sentence) for sentence in data["Description"] if pd.notna(sentence)]

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def get_embedding(sentences):
  # # Load model from HuggingFace Hub
  tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
  model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')


  # Tokenize sentences
  encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

  # Compute token embeddings
  with torch.no_grad():
      model_output = model(**encoded_input)

  # Perform pooling
  sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

  # Normalize embeddings
  sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

  print("Sentence embeddings:")
  print(sentence_embeddings)

  return sentence_embeddings


In [ ]:
sentence_embeddings = get_embedding(sentences)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Sentence embeddings:
tensor([[ 0.0445, -0.0164,  0.0509,  ..., -0.0493, -0.0422, -0.0372],
        [ 0.0274, -0.0019, -0.0267,  ..., -0.0945, -0.0663,  0.0242],
        [ 0.0290,  0.0037,  0.0174,  ..., -0.1056, -0.0760,  0.0496],
        ...,
        [ 0.0283,  0.0101,  0.0485,  ..., -0.0309, -0.1203, -0.0040],
        [-0.0083, -0.0401,  0.0008,  ..., -0.0229,  0.0006, -0.0105],
        [ 0.0530, -0.0130, -0.0223,  ..., -0.0774, -0.0360,  0.0041]])


In [ ]:
!pip3 install -U xformers --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 MB 15.8 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer

query_prompt = ""#"Instruct: Given a curriculum, classify respect in which department the worker corresponds in the company. For example: HR, Finance, Software, etc.\nQuery: "

query_sentences = [query_prompt + query for query in sentences]

# ！The default dimension is 1024, if you need other dimensions, please clone the model and modify `modules.json` to replace `2_Dense_1024` with another dimension, e.g. `2_Dense_256` or `2_Dense_8192` !
# on gpu
model = SentenceTransformer("dunzhang/stella_en_400M_v5", trust_remote_code=True).cuda()
# you can also use this model without the features of `use_memory_efficient_attention` and `unpad_inputs`. It can be worked in CPU.
# model = SentenceTransformer(
#     "dunzhang/stella_en_400M_v5",
#     trust_remote_code=True,
#     device="cpu",
#     config_kwargs={"use_memory_efficient_attention": False, "unpad_inputs": False}
# )
sentence_embeddings = model.encode(query_sentences)

print("Sentence embeddings:")
print(sentence_embeddings)


Some weights of the model checkpoint at dunzhang/stella_en_400M_v5 were not used when initializing NewModel: ['new.pooler.dense.bias', 'new.pooler.dense.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Sentence embeddings:
[[ 0.01411574 -0.34646657 -1.1120074  ...  0.19955876  0.889077
  -0.6775323 ]
 [ 0.06555311 -0.3793232  -0.8226563  ... -0.21754721  0.9338673
  -0.9823674 ]
 [ 0.13714845 -0.379117   -1.1035001  ...  0.13655357  0.46189743
  -0.47446653]
 ...
 [-0.12116702 -0.6514239  -1.2010286  ... -0.21927272  0.47766072
  -0.5496359 ]
 [ 0.64769006 -0.9212938  -0.4968567  ... -0.5020037   0.704821
  -1.206464  ]
 [-0.18061844 -0.37391627 -0.5758207  ... -0.32954165  0.3536765
  -0.77224356]]


In [ ]:
# Save embeddings
df = pd.DataFrame(sentence_embeddings)

df.to_csv('description_embeddings.csv', index=False, header=False)

print("Embeddings saved to 'embeddings.csv'")

Embeddings saved to 'embeddings.csv'


In [ ]:
# Inicializamos instancia TSNE
perplexity_tsne = 15
tsne2d = TSNE(n_components = 2, random_state = 0, perplexity=perplexity_tsne)

# Ajustar el espacio con los puntos de nuestro dataset
proyection_tsne2d = tsne2d.fit_transform(sentence_embeddings)

In [ ]:
# Visualizar
labels = [str(sentence) for sentence in data["Job Title"] if pd.notna(sentence)]
fig = px.scatter(proyection_tsne2d,
                    x=0,y=1,
                    color = labels,
                    labels={'color':'label'}
                    )

fig.show()